# Daily Traffic Counts
Questions:
- Can we get time stamp of the image?
- How can we get live speed data?

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset

%matplotlib inline

/home/ah2719/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Global Variables

In [ ]:
IMG_SIZE = 1024
VEHICLE_DETECTION_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/vehicle_count_detection.txt'
GROUND_TRUTH_DATA_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/processed/ground_truth_data_processed.csv'
LIVE_SPEED_DATA_PATH = ""
ROAD_TYPE_PATH = ""
ROAD_WIDTH_PATH = ""

## Ground Truth Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

In [ ]:
# Get Ground Truth Data

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Neural Network Model

In [ ]:
class CustomDataset(Dataset):
    def __init__(self):
        self.labels = pd.read_csv(GROUND_TRUTH_DATA_PATH)
        self.vehicle_count = pd.read_csv(VEHICLE_DETECTION_COUNT_PATH)
        self.live_speed_data = pd.read_csv(LIVE_SPEED_DATA_PATH) # Or speed limit if live speed data not available
        self.road_type = pd.read_csv(ROAD_TYPE_PATH)
        self.road_width = pd.read_csv(ROAD_WIDTH_PATH)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        vehicle_count = self.vehicle_count.iloc[idx,1]
        live_speed_data = self.live_speed_data.iloc[idx,1] # Or speed limit if live speed data not available
        road_type = self.road_type.iloc[idx,1]
        road_width = self.road_width.iloc[idx,1]
        label = self.labels.iloc[idx, 1]
        inputs = np.array([vehicle_count, live_speed_data, road_type, road_width])
        inputs = pd.DataFrame(inputs, columns=['vehicle_count', 'live_speed_data', 'road_type', 'road_width'])
        return inputs, label

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 1),
            nn.ReLU(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
learning_rate = 1e-3
batch_size = 12
epochs = 5

loss_fn = nn.MSELoss()